In [10]:
import torch
import supervision as sv
import cv2
from ultralytics import YOLO
import numpy as np
print(torch.__version__)        # should show +cu128
print(torch.version.cuda)       # should match runtime CUDA version
print(torch.cuda.is_available())  # True if GPU detected

2.9.1+cu128
12.8
True


In [11]:
VIDEO_PATH = "../VIDEOSYOLO/video3.mp4"

model = YOLO("yolo11n.pt")

frames_generator = sv.get_video_frames_generator(
    source_path=VIDEO_PATH,
)

POLYGON= np.array([[230.55225148683093, 2.463891248937978],
 [322.9906542056075, 108.49617672047579],
 [383.89124893797793, 229.20985556499576],
 [365.94732370433303, 349.37977909940525],
 [438.266779949023, 246.06627017841973],
 [544.2990654205607, 165.0467289719626],
 [640, 129.1588785046729],
 [636.1937128292269, 186.2531860662702],
 [602.480883602379, 212.3534409515718],
 [541.5802888700085, 360],
 [258.2837723024639, 358.6236193712829],
 [275.6839422259983, 135.6839422259983],
 [232.18351741716228, 49.226847918436704],
 [247.95242141036533, 113.93372982158029],
 [218.58963466440102, 359.7111299915038],
 [0, 356.4485981308411],
 [0, 294.4604927782498],
 [156.05777400169924, 96.53355989804588],
 [172.9141886151232, 0.2888700084961767]], dtype=np.int32)

LINE_1_START = sv.Point(12, 287)
LINE_1_END   = sv.Point(236, 294)

LINE_2_START = sv.Point(257, 300)
LINE_2_END   = sv.Point(361, 296)

LINE_3_START = sv.Point(414, 294)
LINE_3_END   = sv.Point(551, 328)

LINE_ZONE_1 = sv.LineZone(
    start=LINE_1_START,
    end=LINE_1_END,
    triggering_anchors=(sv.Position.CENTER,)
)

LINE_ZONE_2 = sv.LineZone(
    start=LINE_2_START,
    end=LINE_2_END,
    triggering_anchors=(sv.Position.CENTER,)
)

LINE_ZONE_3 = sv.LineZone(
    start=LINE_3_START,
    end=LINE_3_END,
    triggering_anchors=(sv.Position.CENTER,)
)


CLASSES = [2,3,5,7]


polygon_zone = sv.PolygonZone(POLYGON, triggering_anchors=(sv.Position.CENTER,))

box_annotator = sv.BoxAnnotator()

label_annotator = sv.LabelAnnotator(text_padding=2)

tracker = sv.ByteTrack(minimum_consecutive_frames=3)
tracker.reset

trace_annotator = sv.TraceAnnotator(trace_length=30)

line_zone_annotator = sv.LineZoneAnnotator(
    text_orient_to_line=True,
    text_scale=0.6,
    custom_in_text="dentro",
    custom_out_text="fuera"
    )

LINE_ZONE_ANNOTATOR_MULTICLASS = sv.LineZoneAnnotatorMulticlass(
    text_scale=0.8,
    text_thickness=2,
    table_margin=20
)

for idx, frame in enumerate(frames_generator):
    result = model(frame, device="cuda", verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #print(detections)
    detections = detections[polygon_zone.trigger(detections)]
    detections = detections[np.isin(detections.class_id, CLASSES)]
    
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id}"
        for tracker_id
        in detections.tracker_id
    ]

    LINE_ZONE_1.trigger(detections=detections)
    LINE_ZONE_2.trigger(detections=detections)
    LINE_ZONE_3.trigger(detections=detections)
    
    annotated_frame = frame.copy()
    annotated_frame = sv.draw_polygon(scene=annotated_frame, polygon=POLYGON, color=sv.Color.YELLOW, thickness=2)
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame,
        detections=detections
    )
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels
    )
    annotated_frame = trace_annotator.annotate(
        scene=annotated_frame,
        detections=detections
    )
    annotated_frame = line_zone_annotator.annotate(
        annotated_frame, line_counter=LINE_ZONE_1
    )
    annotated_frame = line_zone_annotator.annotate(
        annotated_frame, line_counter=LINE_ZONE_2
    )
    annotated_frame = line_zone_annotator.annotate(
        annotated_frame, line_counter=LINE_ZONE_3   
    )
    annotated_frame = LINE_ZONE_ANNOTATOR_MULTICLASS.annotate(
        annotated_frame,
        line_zones= [LINE_ZONE_1]
    )


    cv2.imshow("Annotated Frame", annotated_frame)

    # salir con 'q'
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is deprecated: `overlay_image` function is deprecated and will be removed in `supervision-0.32.0`. Use `draw_image` instead.
SupervisionWarnings: overlay_image is de